# Exercises

## Exercise 1
Write yet another function that solves for element n of the Fibonacci sequence,
using a technique of your own design. Write unit tests that evaluate its correctness
and performance relative to the other versions in this chapter.

### Solution
The Fibonacci numbers have a closed-form solution (https://en.wikipedia.org/wiki/Fibonacci_number#Closed-form_expression):

$F_n = \left \lfloor \frac{\phi^n}{\sqrt{5}}+\frac{1}{2} \right \rfloor$, with

$\phi = \frac{1+\sqrt{5}}{2}$

In [125]:
import math
def fib_closed(n: int) -> int:
    phi = (1 + math.sqrt(5))/2
    return math.floor((phi**n)/math.sqrt(5)+(1/2))

Unit tests

In [126]:
from fib5 import fib5 
from fib4 import fib4
import timeit
for i in [0,1,5,25, 50,75, 100]:
    print(f"fib5({i}) = {fib5(i)}, fib_closed({i})={fib_closed(i)}")

fib5(0) = 0, fib_closed(0)=0
fib5(1) = 1, fib_closed(1)=1
fib5(5) = 5, fib_closed(5)=5
fib5(25) = 75025, fib_closed(25)=75025
fib5(50) = 12586269025, fib_closed(50)=12586269025
fib5(75) = 2111485077978050, fib_closed(75)=2111485077978055
fib5(100) = 354224848179261915075, fib_closed(100)=354224848179263111168


Seems the closed form solution suffers from rounding errors when calculating very big fibonacci numbers.

In [117]:
from timeit import default_timer as timer

In [121]:
from typing import Callable
def fib_perf(fib: Callable[[int],int], n:int = 50, repeats:int = 1000000)->float:
    start = timer()
    for i in range(repeats):
        fib(n)
    end = timer()
    return(end - start)

In [122]:
for n in [5, 10, 25, 50]:
    print(f"fib_closed speedup for fib({n}): {fib_perf(fib5, n)/fib_perf(fib_closed, n)}")

fib_closed speedup for fib(5): 0.5656257381690774
fib_closed speedup for fib(10): 0.7970060094147272
fib_closed speedup for fib(25): 1.9666871906484276
fib_closed speedup for fib(50): 3.391618790332616


## Exercise 2
You saw how the simple int type in Python can be used to represent a bit string.
Write an ergonomic wrapper around int that can be used generically as a
sequence of bits (make it iterable and implement `__getitem__()` ). Reimplement
CompressedGene, using the wrapper.

In [144]:
from sys import getsizeof
class BitString:
    """
    Provides an indexable and iterable BitString with configurable width
    """
    def __init__(self, width:int=1)->None:
        self.bit_string = 0
        self.width = width
        self.item_max = (1<<self.width)-1
        self.length = 0
    
    def __getitem__(self, i:int)->int:
        if i>=self.length:
            raise IndexError('Out of bounds')
        return self.bit_string>>(i*self.width) & self.item_max        
    
    def __setitem__(self, i:int, val:int)->None:
        if val>self.item_max:
            raise Exception(f"Value {val} has more than {self.width} bits.")
        if i+1>self.length:
            self.length = i+1
        self.bit_string = self.bit_string | val<<(i*self.width)
        
    def __str__(self)->str:
        return f"BitString with width {self.width}: {[x for x in self]}"
    
    def __len__(self)->int:
        return self.length
    
    def __sizeof__(self)->int:
        return getsizeof(self.bit_string)

In [145]:
bs = BitString(width=2)
bs[1] = 1
bs[4] = 3

In [146]:
str(bs)

'BitString with width 2: [0, 1, 0, 0, 3]'

In [147]:
len(bs)

5

In [148]:
getsizeof(bs)

52

In [149]:
for i in bs:
    print(f"{i}")

0
1
0
0
3


In [150]:
bs[2]=4

Exception: Value 4 has more than 2 bits.

In [151]:
class CompressedGeneBS:
    def __init__(self, gene: str) -> None:
        self._compress(gene)

    def _compress(self, gene: str) -> None:
        self.bit_string = BitString(width=2)
        i = 0
        for nucleotide in gene.upper():
            if nucleotide == "A":  # change last two bits to 00
                self.bit_string[i] = 0b00
            elif nucleotide == "C":  # change last two bits to 01
                self.bit_string[i] = 0b01
            elif nucleotide == "G":  # change last two bits to 10
                self.bit_string[i] = 0b10
            elif nucleotide == "T":  # change last two bits to 11
                self.bit_string[i] = 0b11
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))
            i += 1

    def decompress(self) -> str:
        gene: str = ""
        for bits in self.bit_string:
            if bits == 0b00:  # A
                gene += "A"
            elif bits == 0b01:  # C
                gene += "C"
            elif bits == 0b10:  # G
                gene += "G"
            elif bits == 0b11:  # T
                gene += "T"
            else:
                raise ValueError("Invalid bits:{}".format(bits))
        return gene

    def __str__(self) -> str:  # string representation for pretty printing
        return self.decompress()

In [152]:
from sys import getsizeof
original: str = "TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA" * 100
print("original is {} bytes".format(getsizeof(original)))
compressed: CompressedGeneBS = CompressedGeneBS(original)  # compress
print("compressed is {} bytes".format(getsizeof(compressed.bit_string)))
print(compressed)  # decompress
print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 8649 bytes
compressed is 2344 bytes
TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGA